In [1]:
ct_model_7='/home/nada.saadi/CTPET/hecktor2022_cropped/7thctchannel/7thCTonly_channel.pth'
pt_model_7='/home/nada.saadi/CTPET/hecktor2022_cropped/7thPETchannelonly/7thPETonly_channel.pth'

In [2]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from glob import glob
# import wandb

import monai
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference
from monai import transforms

from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR, UNETR, SegResNet

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai import data


from monai.utils import first, set_determinism
from sklearn.model_selection import train_test_split
import json


from monai.transforms import apply_transform
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.data import DataLoader, Dataset, decollate_batch
from tqdm import tqdm
import json
import torch

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections.abc import Sequence

from unetr import CustomedUNETR
# Load the pretrained weights
#pretrained_weights = torch.load('/home/nada.saadi/CTPET/hecktor2022_cropped/4centers-ctonly/4centers-ctonly.pth')


model_ct = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072, 
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

model_pet = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072, 
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


In [4]:
model_ct.load_state_dict(torch.load(ct_model_7))
model_pet.load_state_dict(torch.load(pt_model_7))

    

model_ct.load_state_dict(torch.load(ct_model_7), strict=False)
model_pet.load_state_dict(torch.load(pt_model_7), strict=False)

model_ct.eval() 
model_pet.eval()


UNETR(
  (vit): ViT(
    (patch_embedding): PatchEmbeddingBlock(
      (patch_embeddings): Sequential(
        (0): Rearrange('b c (h p1) (w p2) (d p3) -> b (h w d) (p1 p2 p3 c)', p1=16, p2=16, p3=16)
        (1): Linear(in_features=4096, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (blocks): ModuleList(
      (0-11): 12 x TransformerBlock(
        (mlp): MLPBlock(
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (linear2): Linear(in_features=3072, out_features=768, bias=True)
          (fn): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (drop2): Dropout(p=0.0, inplace=False)
        )
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SABlock(
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_q): Linear(in_features=768, out_features=768, bias=False)
          (proj_k): Linear(in_features=

In [5]:
ct_data_7='/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_CT_train_new.json'
Pt_data_7='/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_PET_train_new.json'

In [6]:
import os
import json
import random
from glob import glob

def generate_paths(patient_id):
    base_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data'
    return {
        'id': os.path.join(base_dir, patient_id),
        'ct': os.path.join(base_dir, patient_id, f"{patient_id}_ct.nii.gz"),
        'seg': os.path.join(base_dir, patient_id, f"{patient_id}_gt.nii.gz")
    }

# Assign each data entry to a random fold
all_data = []
num_folds = 5

for file_dir in sorted(glob('data/*')):
    patient_id = file_dir.split('/')[-1]
    # Check if the file belongs to the MDA center
    if patient_id.startswith("HMR-"):
        entry = generate_paths(patient_id)
        entry['fold'] = random.randint(1, num_folds) - 1
        all_data.append(entry)

# Compile data into a JSON structure
data_json = {"training": all_data}

# Save to JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_CT_train_new.json"
with open(json_file_path, 'w') as f:
    json.dump(data_json, f, indent=4)

print(f"JSON file created at {json_file_path}")

JSON file created at /home/nada.saadi/CTPET/hecktor2022_cropped/HMR_CT_train_new.json


In [7]:
import glob

hmr_ct_files = glob.glob('/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-*/HMR-*_ct.nii.gz')
num_files = len(hmr_ct_files)

print(f"Number of files in HMR CT samples: {num_files}")


Number of files in HMR CT samples: 17


In [8]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir_ct = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_CT_train_new.json"

In [9]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [24]:
import json
train_files, validation_files = datafold_read(datalist=json_dir_ct, basedir=data_dir, fold=4)
len(train_files), len(validation_files)
# Path for the new JSON file
ct_validation_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json'

# Save the validation files to a JSON file
with open(ct_validation_json_path, 'w') as file:
    json.dump(validation_files, file, indent=4)

print(f"Validation files have been saved to {ct_validation_json_path}")


Validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json


In [25]:
len(validation_files)
hmr_ct_validation_files = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json'

In [26]:
len(validation_files)

4

In [27]:
import json

# Path to the JSON file
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json'

# Read the JSON file
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Count the number of elements
num_elements = len(data)

# Print the number of elements
print(f"The number of elements in hmr_ct_validation_files is: {num_elements}")

The number of elements in hmr_ct_validation_files is: 4


In [28]:
with open('/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json', 'r') as file:
    hmr_ct_validation_files = json.load(file)

print(hmr_ct_validation_files)

[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_gt.nii.gz', 'fold': 4}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_gt.nii.gz', 'fold': 4}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_gt.nii.gz', 'fold': 4}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-029', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-029/HMR-029_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-029/HMR-029_gt.nii.gz', 'fold': 4}]


In [29]:
import os
import json
import random
from glob import glob

def generate_paths(patient_id):
    base_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data'
    return {
        'id': os.path.join(base_dir, patient_id),
        'pt': os.path.join(base_dir, patient_id, f"{patient_id}_pt.nii.gz"),
        'seg': os.path.join(base_dir, patient_id, f"{patient_id}_gt.nii.gz")
    }

# Assign each data entry to a random fold
all_data = []
num_folds = 5

for file_dir in sorted(glob('data/*')):
    patient_id = file_dir.split('/')[-1]
    # Check if the file belongs to the MDA center
    if patient_id.startswith("HMR-"):
        entry = generate_paths(patient_id)
        entry['fold'] = random.randint(1, num_folds) - 1
        all_data.append(entry)

# Compile data into a JSON structure
data_json = {"training": all_data}

# Save to JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_PET_train_new.json"
with open(json_file_path, 'w') as f:
    json.dump(data_json, f, indent=4)

print(f"JSON file created at {json_file_path}")

JSON file created at /home/nada.saadi/CTPET/hecktor2022_cropped/HMR_PET_train_new.json


In [30]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir_pt = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_PET_train_new.json"

In [31]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [68]:
import json
train_files, validation_files = datafold_read(datalist=json_dir_pt, basedir=data_dir, fold=2)
len(train_files), len(validation_files)
# Path for the new JSON file
validation_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_pt_validation_files.json'

# Save the validation files to a JSON file
with open(validation_json_path, 'w') as file:
    json.dump(validation_files, file, indent=4)

print(f"Validation files have been saved to {validation_json_path}")

Validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/hmr_pt_validation_files.json


In [69]:
import glob

hmr_pt_files = glob.glob('/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-*/HMR-*_pt.nii.gz')
num_files = len(hmr_pt_files)

print(f"Number of files in HMR PET samples: {num_files}")

Number of files in HMR PET samples: 17


In [70]:
len(validation_files)

4

In [71]:
hmr_pt_validation_files='/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_pt_validation_files.json'

In [73]:
import json

# Path to the JSON file
json_file_path = hmr_pt_validation_files

# Read the JSON file
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Print the first 5 elements
for i in range(3):
    print(data[i])

{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_gt.nii.gz', 'fold': 2}
{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_gt.nii.gz', 'fold': 2}
{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_gt.nii.gz', 'fold': 2}


In [38]:
import json

# Path to the existing JSON files
ct_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json'
pet_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_pt_validation_files.json'

# Path for the new JSON files
ct_new_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_modified_validation_files.json'
pet_new_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_pt_modified_validation_files.json'

# Read the existing JSON files
with open(ct_json_path, 'r') as ct_file, open(pet_json_path, 'r') as pet_file:
    ct_data = json.load(ct_file)
    pet_data = json.load(pet_file)

# Remove the 'id' and 'fold' keys from the data
ct_modified_data = [{k: v for k, v in item.items() if k not in ['id', 'fold']} for item in ct_data]
pet_modified_data = [{k: v for k, v in item.items() if k not in ['id', 'fold']} for item in pet_data]

# Save the modified data to new JSON files
with open(ct_new_json_path, 'w') as ct_new_file, open(pet_new_json_path, 'w') as pet_new_file:
    json.dump(ct_modified_data, ct_new_file, indent=4)
    json.dump(pet_modified_data, pet_new_file, indent=4)

print(f"Modified CT validation files have been saved to {ct_new_json_path}")
print(f"Modified PET validation files have been saved to {pet_new_json_path}")

Modified CT validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_modified_validation_files.json
Modified PET validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/hmr_pt_modified_validation_files.json


In [39]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [40]:
val_transforms_ct = Compose(
    [
        LoadImaged(keys=["ct",  "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct",], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        ConcatItemsd(keys=[ "ct"], name="ct"),
    ]
)
def create_dataloader(data, transforms, batch_size=2, shuffle=True):
    # Create CacheDataset with the reformatted data
    dataset = Dataset(data=data, transform=transforms)

    # Create DataLoader
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)
val_loader = create_dataloader(ct_modified_data, val_transforms_ct, shuffle=False)

In [41]:
val_transforms_pt = Compose(
    [
        LoadImaged(keys=[ "pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=[ "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=[ "pt", "seg"], axcodes="PLS"),
        NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["pt"]),
        ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        ConcatItemsd(keys=["pt"], name="pt"),
    ]
)
def create_dataloader(data, transforms, batch_size=2, shuffle=True):
    # Create CacheDataset with the reformatted data
    dataset = Dataset(data=data, transform=transforms)

    # Create DataLoader
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)
val_loader = create_dataloader(pet_modified_data, val_transforms_pt, shuffle=False)

In [42]:
import io

with open(ct_json_path, 'r') as f:
    test_data = json.load(f)

test_loader = DataLoader(
    dataset=Dataset(data=test_data, transform=val_transforms_ct),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)

In [43]:
len(test_data)

4

In [74]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir_ctpt = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ctpt_train_new.json"

In [83]:
import json

# Path to the existing JSON files
# Check if ctpt_data is a list of dictionaries
if isinstance(ctpt_data, list) and all(isinstance(item, dict) for item in ctpt_data):
    # Remove the 'id' and 'fold' keys from the data
    ctpt_modified_data = [{k: v for k, v in item.items() if k not in ['id', 'fold']} for item in ctpt_data]
else:
    print("ctpt_data is not a list of dictionaries.")


# Save the modified data to new JSON files
with open(ctpt_new_json_path, 'w') as ct_new_file, open(ctpt_new_json_path, 'w') as ctpt_new_file:
    json.dump(ctpt_modified_data, ct_new_file, indent=4)
  
    # Check if ctpt_data is a list of dictionaries
    if not isinstance(ctpt_data, list) or not all(isinstance(item, dict) for item in ctpt_data):
        ctpt_data = [ctpt_data]  # Convert single dictionary to a list of dictionaries

print(f"Modified CT validation files have been saved to {ctpt_new_json_path}")


Modified CT validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ctpt_modified_validation_files.json


In [91]:
ctpt_data

[{'training': [{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001',
    'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_ct.nii.gz',
    'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_pt.nii.gz',
    'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-001/HMR-001_gt.nii.gz',
    'fold': 3},
   {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004',
    'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_ct.nii.gz',
    'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_pt.nii.gz',
    'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-004/HMR-004_gt.nii.gz',
    'fold': 4},
   {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005',
    'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005/HMR-005_ct.nii.gz',
    'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-005/HMR-005_pt.nii.gz',
    'seg': '/home/nada.saadi/MIS-FM/heck

In [90]:
# Specify the path to the JSON file
json_file_path = ctpt_new_json_path

# Read the JSON file
with open(json_file_path, "r") as json_file:
    ctpt_data = json.load(json_file)

# Assuming the structure is always like the one provided,
# where ctpt_data is a list containing a single dictionary,
# and 'training' is a key in that dictionary:

# Access the 'training' key in the first dictionary of the list
training_data = ctpt_data[0]['training']
val_ctpt_data = ctpt_data[0]['validation']
# Count the elements in training_data
training_data_count = len(training_data)
val_data_count = len(val_ctpt_data)

# Print the count
print("Number of samples in 'training':", training_data_count)
print("Number of samples in 'validation':", val_data_count)


KeyError: 'validation'

In [93]:
import json

# Specify the path to the JSON file
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ctpt_modified_validation_files.json'  # Adjust this to your actual file path

# Read the JSON file
with open(json_file_path, "r") as json_file:
    ctpt_data = json.load(json_file)

# Filter samples where 'fold' is 2 to create a validation set
validation_data = []
for group in ctpt_data:
    for sample in group['training']:
        if sample['fold'] == 2:
            validation_data.append(sample)

# Count the number of validation files
validation_count = len(validation_data)
print("Number of validation files:", validation_count)

# Save the filtered data to a new JSON file
new_json_file_path = 'hmr_ctpt_val.json'  # Name of the new JSON file
with open(new_json_file_path, "w") as new_json_file:
    json.dump(validation_data, new_json_file, indent=4)

print(f"Validation data saved to {new_json_file_path}.")


Number of validation files: 5
Validation data saved to hmr_ctpt_val.json.


In [94]:
import json

# Specify the path to the JSON file
json_file_path = new_json_file_path

# Read the JSON file
with open(json_file_path, "r") as json_file:
    data = json.load(json_file)

# Print the data
print(data)


[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_gt.nii.gz', 'fold': 2}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_gt.nii.gz', 'fold': 2}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_gt.nii.gz', 'fold': 2}, {'i

In [88]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [92]:
train_files_ctpt, validation_files_ctpt = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
len(train_files_ctpt), len(validation_files_ctpt)

(13, 4)

In [ ]:
import json

# Define the path of the new JSON file
new_json_path = "/path/to/new_json_file.json"

# Save the ctpt validation files to the new JSON file
with open(new_json_path, "w") as f:
    json.dump(ctpt_data, f)


In [55]:
import json

# Define the path of the previous validation files JSON
previous_json_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ctpt_train_new.json"

# Define the path of the new JSON file
new_json_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/val_ctpt_hmr.json"

# Load the previous validation files JSON
with open(previous_json_path, "r") as f:
    previous_json = json.load(f)

# Save the previous validation files as a new JSON file
with open(new_json_path, "w") as f:
    json.dump(previous_json, f)


In [97]:
import json

# Specify the path to the JSON file
json_file_path = new_json_file_path

# Read the JSON file
with open(json_file_path, "r") as json_file:
    data_ctpt = json.load(json_file)

# Print the data
print(data_ctpt)


[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-013/HMR-013_gt.nii.gz', 'fold': 2}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-020/HMR-020_gt.nii.gz', 'fold': 2}, {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/HMR-021/HMR-021_gt.nii.gz', 'fold': 2}, {'i

In [99]:


test_loader = DataLoader(
    dataset=Dataset(data=test_data, transform=val_transforms_ct),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model_ct.parameters(), lr=1e-4, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

# Dice metric for evaluation
dice_metric_fn = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

#pt_weights='/home/nada.saadi/CTPET/hecktor2022_cropped/4centers-ctonly/4centers-ctonly.pth'
# pt_weights_5='/home/nada.saadi/CTPET/hecktor2022_cropped/5th_ctpt_tokens_ourapproach/5th_center_our_approach.pth'
# state_dict = torch.load(pt_weights_5)
# model_state_dict = model_ct.state_dict()
# for name, param in state_dict.items():
#     if name in model_state_dict:
#         if param.shape == model_state_dict[name].shape:
#             model_state_dict[name] = param

def testing():
    model_ct.eval()
    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs, test_labels = batch_data["ct"].cuda(), batch_data["seg"].cuda()
            
            test_outputs_ct = sliding_window_inference(test_inputs, (96, 96, 96), 4, model_ct)
            
            # Convert outputs and labels to one-hot format for DiceMetric calculation
            test_outputs_ct = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(test_outputs_ct)]
            test_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]

            dice_metric_fn(y_pred=test_outputs_ct, y=test_labels)
            dice_metric_batch_fn(y_pred=test_outputs_ct, y=test_labels)
            
        # After processing all batches, save the outputs
    #ct_save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_output_ct_saved.pt'
    ct_save_path_6thcenter = '/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_test_output_ct_saved.pt'
    torch.save(test_outputs_ct, ct_save_path_6thcenter)
    print(f"Outputs saved at: {ct_save_path_6thcenter}")
    

    mean_dice_test = dice_metric_fn.aggregate().item()
    metric_batch_test = dice_metric_batch_fn.aggregate()
    metric_tumor = metric_batch_test[0].item()
    metric_lymph = metric_batch_test[1].item()

    print(f"Testing - Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
    return mean_dice_test, metric_tumor, metric_lymph
    # Save the weights
testing() 

  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/data/dataset.py", line 112, in __getitem__
    return self._transform(index)
  File "/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/data/dataset.py", line 97, in _transform
    data_i = self.data[index]
KeyError: 0
